In [67]:
from dotenv import load_dotenv
from pyflink.table.confluent import ConfluentSettings, ConfluentTools, ConfluentTableDescriptor
from pyflink.table import TableEnvironment, DataTypes, Schema
from pyflink.table.expressions import *
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table.window import Tumble, Slide, Session
from pyflink.table.udf import udf

In [68]:
from dotenv import load_dotenv
load_dotenv()
# load_dotenv("/Users/dwoodbridge/Class/2025_MSDS682/Examples/msdes682_data_stream_processing_2025/Day11/.env")

True

| +I |                              2 |                           65.1 | 2025-11-21 05:36:41.000 | 2025-11-21 05:36:42.000 |
| +I |                              3 |                           65.2 | 2025-11-21 05:36:42.000 | 2025-11-21 05:36:43.000 |
| +I |                              2 |                           65.1 |                    1 | 2025-11-21 05:36:39.000 | 2025-11-21 05:36:42.000 |
| +I |                              1 |                           65.2 | 2025-11-21 05:36:43.000 | 2025-11-21 05:36:44.000 |
| +I |                              4 |                           65.2 | 2025-11-21 05:36:44.000 | 2025-11-21 05:36:45.000 |
| +I |                              5 |                           65.2 | 2025-11-21 05:36:45.000 | 2025-11-21 05:36:46.000 |
| +I |                              4 |                           65.2 |                    1 | 2025-11-21 05:36:42.000 | 2025-11-21 05:36:45.000 |
| +I |                              1 |                           65.2 |       

In [69]:
import os
print(os.getenv("CLOUD_REGION"))

us-west1


In [70]:
settings = ConfluentSettings.from_global_variables()
settings = settings.in_streaming_mode()
env = TableEnvironment.create(settings)

In [71]:
env.use_catalog("msds682")
env.use_database("msds682")
env.execute_sql("""DROP TABLE IF EXISTS `system.usage.new`""")

In [72]:
env.execute_sql("""
    CREATE TABLE IF NOT EXISTS `system.usage.new` (
        id STRING,
        cpu_usage FLOAT,
        cpu_stats ARRAY<FLOAT>,
        memory_usage FLOAT,
        `timestamp` TIMESTAMP_LTZ(3),
        WATERMARK FOR `timestamp` AS `timestamp` - INTERVAL '5' SECOND
    ) WITH (
        'value.format'='avro-registry',
        'connector' = 'confluent'
    )
""")

In [73]:
table = env.from_path("`system.usage.new`")

In [ ]:
# from pyflink.table.expressions import col, lit,interval

windowed_table = table.window(Tumble.over(lit(3).second)
            .on(col("timestamp"))
            .alias("w")           
)
result = windowed_table.group_by(
    col("w"), 
    col("id")
).select(
    col("id"),
    col("memory_usage").min.alias("min_memory_usage"),
    col("memory_usage").max.alias("max_memory_usage"), # Access 'max' as a property
    col("memory_usage").count.alias("record_ct"),
    col("w").start.alias("window_start"),              
    col("w").end.alias("window_end")                   
)

| +I |                              5 |                           65.2 |                    1 | 2025-11-21 05:36:45.000 | 2025-11-21 05:36:48.000 |
| +I |                              4 |                           65.2 |                    1 | 2025-11-21 05:36:45.000 | 2025-11-21 05:36:48.000 |
| +I |                              4 |                           65.0 |                    1 | 2025-11-21 05:36:48.000 | 2025-11-21 05:36:51.000 |
| +I |                              5 |                           65.2 |                    1 | 2025-11-21 05:36:48.000 | 2025-11-21 05:36:51.000 |
| +I |                              2 |                           65.1 |                    1 | 2025-11-21 05:36:48.000 | 2025-11-21 05:36:51.000 |
| +I |                              3 |                           65.4 |                    1 | 2025-11-21 05:36:51.000 | 2025-11-21 05:36:54.000 |
| +I |                              4 |                           65.2 |                           65.2 |       

In [75]:
ConfluentTools.print_changelog(result)

| +I |                              4 |                           65.5 | 2025-11-21 05:37:13.000 | 2025-11-21 05:37:14.000 |
| +I |                              3 |                           65.5 | 2025-11-21 05:37:14.000 | 2025-11-21 05:37:15.000 |
| +I |                              4 |                           65.6 | 2025-11-21 05:37:15.000 | 2025-11-21 05:37:16.000 |
| +I |                              4 |                           65.5 |                    1 | 2025-11-21 05:37:13.000 | 2025-11-21 05:37:14.000 |
| +I |                              3 |                           65.5 |                    1 | 2025-11-21 05:37:14.000 | 2025-11-21 05:37:15.000 |
| +I |                              4 |                           65.6 |                    1 | 2025-11-21 05:37:15.000 | 2025-11-21 05:37:16.000 |
| +I |                              4 |                           65.5 |                    1 | 2025-11-21 05:37:12.000 | 2025-11-21 05:37:15.000 |
| +I |                           

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/msds682-new/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/msds682-new/lib/python3.11/site-packages/py4j/java_gateway.py", line 1217, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/envs/msds682-new/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

| +I |                              2 |                           65.7 |                           65.7 |                    1 | 2025-11-21 05:37:54.000 | 2025-11-21 05:37:57.000 |
